### **Documentation for the `LNG_pricing_sheet_construction.py`**

**Three core functions that help us obtain our curves from barchart**

* `parse_barchart_symbols` (line 29)
* `fetch_barchart_prices` (line 55)
* `clean_and_format_df` (line 111)

---
### Function: `parse_barchart_symbols`  *(line 29 onwards)*

**Purpose**  
Build the list of monthly futures symbols for a given base code (e.g. `NG`, `INK`, `JKM`) from a start date out to a forward horizon.  
Example outputs: `["NGU25", "NGV25", ...]`.

**Inputs**
- `base_code` *(str)*: Contract root, e.g. `"NG"`, `"QA"`, `"INK"`, `"JKM"`.
- `start_date` *(datetime)*: Usually `datetime.today()`.
- `years_forward` *(int)*: How far ahead to generate.

**Output**
- `list[str]`: Monthly symbols in chronological order, plus **all 12 months** of the final end year.

**How it works**
1. Normalises `start_date` to the first of the month.
2. Steps month‑by‑month until (but not including) `end_year`.
3. Appends **Jan–Dec** for `end_year` so you always finish on a full calendar year.
4. Uses the standard month code map: `F,G,H,J,K,M,N,Q,U,V,X,Z`.

**Notes**
- Cash contracts (`Y00`) are not produced here — add them via `commodity_config` (handled at **line 142** and documented further below under the pull config section).
- If you change the horizon logic, keep the “full final year” behaviour unless you explicitly want to stop mid‑year.

---

### Function: `fetch_barchart_prices`  *(line 55 onwards)*

**Purpose**  
Fetch the latest available flat price for each symbol from Barchart (overview page), returning a tidy two‑column frame.

**Inputs**
- `symbols` *(list[str])*: e.g. `["NGU25", "NGV25", ...]` (or includes a `Y00` cash symbol).
- `base_url` *(str, optional)*: Defaults to `https://www.barchart.com/futures/quotes/{}/overview`.

**Output**
- `pd.DataFrame` with columns:  
  - `symbol`  
  - `last_price` *(float, rounded to 4dp)*

**How it works**
1. (Gently) rate‑limits requests with `sleep(0.5)` to avoid throttling.
2. GETs each symbol page with a desktop UA and 10s timeout.
3. Parses the HTML, finds `<div class="symbol-header-info" data-ng-init="init({...})">`.
4. Extracts JSON from the `init(...)` block, takes `lastPrice`, strips any trailing non‑numeric, casts to `float`.
5. Skips symbols with missing/placeholder prices; logs simple `[WARN]/[ERROR]` messages.

**Notes**
- Resilient to hiccups: bad pages or timeouts don’t kill the run; they’re skipped.
- If Barchart changes the page structure, the `data-ng-init` parsing may need updating.
- You can tune or remove `sleep(0.5)` if it becomes too slow (at your throttle risk).

---
### Function: `clean_and_format_df`  *(line 111 onwards)*

**Purpose**  
Turn raw symbol/price rows into a compact two‑column table ready for merging/spread calcs. Generates a human‑readable **`label`** like `Aug'25`, or **`Cash`** for `Y00`.

**Inputs**
- `df_raw` *(DataFrame)*: Must contain `symbol` and `last_price`.
- `label` *(str)*: Column name to assign to prices (e.g. `"Henry Hub ($/MMBtu)"`, `"Brent ($/bbl)"`, `"JKM ($/MMBtu)"`).

**Output**
- `pd.DataFrame` with columns:
  - `label`  → `"Cash"` for Y00, otherwise `"Mmm'YY"` (e.g. `Aug'25`)
  - `<label>` → price column named exactly as passed in

**How it works**
1. Uses regex to extract the **month code** and **yy** from the `symbol` (supports 2–3 char roots).
2. Maps month code → month name via `month_map`.
3. Converts yy → full year (`2000 + yy`).
4. Detects any `Y00` cash symbols:
   - sets `label = "Cash"`,
   - sets month/year to **today** (used only for labelling; you later drop front‑month label anyway).
5. Renames `last_price` → your provided `label`, returns `["label", label]`.

**Notes**
- The main loop later **drops** the current front‑month label (built from today) to avoid mixing with the Cash row.
- If you ever add symbols with different shapes, the regex (`[A-Z]{2,3}([FGHJKMNQUVXZ])\d{2}`) keeps things robust.


### Minimal usage pattern

```python
# 1) Build symbols (plus add cash if configured)
symbols = parse_barchart_symbols(base_code="NG", start_date=datetime.today(), years_forward=12)

# 2) Fetch prices
raw_df = fetch_barchart_prices(symbols)

# 3) Clean + label for merging
df_ng = clean_and_format_df(raw_df, "Henry Hub ($/MMBtu)")


---
### Pull config and scraping loop (lines 137–189)

This section defines **what to pull** (per commodity) and runs the end-to-end pipeline:  
1) build symbols → 2) fetch prices → 3) clean/label → 4) store per-commodity DataFrame.

**Key variables**
- `today` / `start_date`: anchor for symbol generation and front-month detection.  
- `commodity_config`: one entry per curve we want to pull.

**commodity_config structure** — line 142
- `NG` → Henry Hub (usd/MMBtu), 12 years quoted length on the curve, has cash (`NGY00`)  
- `QA` → Brent (usd/bbl), 8 years, has cash (`QAY00`)  
- `INK` → TTF (usd/MMBtu), 3 years, no cash  
- `JKM` → JKM (usd/MMBtu), 5 years, no cash  
- `NF` → NBP (p/th), 7 years, no cash  

> These horizons reflect where Barchart lists reliable monthly contracts. Beyond these periods, quoted contracts do not exist.

**What the loop does**
For each commodity in `commodity_config`:  
- Start an empty list of symbols.
- If `has_cash` is true, prepend the cash symbol (e.g., `NGY00`, `QAY00`).
- Generate monthly futures symbols with `parse_barchart_symbols(...)`.
- Pull prices with `fetch_barchart_prices(symbols)`.
- Clean/format with `clean_and_format_df(...)`.
- Drop the current **front-month** label (based on today’s date).  
- Save the result into `commodity_data[code]`.

**Outputs created**
- `commodity_data["NG"]` → Henry Hub (usd/MMBtu)  
- `commodity_data["QA"]` → Brent (usd/bbl)  
- `commodity_data["INK"]` → TTF (usd/MMBtu)  
- `commodity_data["JKM"]` → JKM (usd/MMBtu)  
- `commodity_data["NF"]` → NBP (p/th)  

These are then stored in convenience variables for later use:  
- `df_ng`, `df_qa`, `df_ink`, `df_jkm`, `df_nbp`

**How to add a new commodity**
1. Add a new entry in `commodity_config` with:  
   - `label`: output column name (include units)  
   - `base_code`: the Barchart root (2–3 letters)  
   - `years`: how far forward to pull  
   - `has_cash`: True/False  
   - `cash_symbol`: only if `has_cash=True`  (check the website for the correct contract code)
2. The loop will then automatically:
   - generate symbols (plus cash if set),
   - fetch and clean prices,
   - add the cleaned DataFrame to `commodity_data`.
3. Optionally assign it to a variable, e.g. `df_new = commodity_data["NEW"]`.

**Sanity check**
There is a commented “cross checker” that prints the last contract label for each curve.  
Re-enable it if you want to confirm the pull length.

---
### Cable forward curve extraction (lines 194–240)

This section pulls the **GBP/USD forward curve** (“Cable”) to convert NBP from **pence/therm → usd/MMBtu**.


**Process**
1. `fetch_forward_fx_rates(url, tenor_list)` requests the FX Empire page (with a fake User-Agent).  
2. **BeautifulSoup** parses the HTML:  
   - `soup.find('td', string=...)` locates the `<td>` containing the tenor label (e.g. *One Month*).  
   - `find_next_siblings("td", limit=3)` reads the next three cells: bid / ask / mid.  
   - Values are stripped, cast to `float`, and rounded (4dp).  
3. Returns a DataFrame of `[tenor, bid, ask, mid]`.

**Tenors used**  
- From **1 to 11 months ahead**, we pull monthly forwards: `"One Month"`, `"Two Month"`, … `"Eleven Month"`.  
- Beyond 11 months, the curve switches to a yearly granularity: `"One Year"`, `"Two Year"`, … up to `"Five Year"`.  
- Very short-dated contracts (e.g. `"One Week"`, `"Two Week"`, `"Three Week"`) are skipped — they track almost identically to the 1-month tenor and add little value.

**Output**
- `cable_fwd_data` holds the raw table.  
- `df_fx` keeps only `tenor` + `mid`, renamed as **Cable Fwd Rate**.

**Notes**
- If tenor text doesn’t match site labels, you’ll see a `[WARN]` message.  
- Site structure changes (e.g. column order) may break parsing — adjust selectors if needed.  
- These FX forwards are later mapped to NBP contracts by month-offset before conversion (process covered in greater detail below)

---
### Applying the FX forward curve to NBP (lines 241–307)

NBP prices are quoted in **pence/therm**, so we apply the **GBP/USD forward curve** (“Cable”) to convert them into **usd/MMBtu**.  
This ensures that each forward contract is valued consistently in dollar terms.

**Step-by-step process**

1. **Parse contract dates**  
   - Each NBP label (e.g. `Jan'26`) is converted into month/year fields.  
   - This allows us to measure how far forward each contract is from the **front month**.

2. **Identify front month/year**  
   - The very first row of the curve defines the anchor point (`front_year`, `front_month`).  
   - All later contracts are measured relative to this.

3. **Map contracts to FX tenors**  
   - A helper (`map_fx_tenor`) calculates the month difference (`month_diff`) between a contract and the front month.  
   - Mapping rules:  
     - **0–10 months ahead** → `"One Month"` … `"Eleven Month"` (monthly forwards)  
     - **11–22 months** → `"One Year"`  
     - **23–34 months** → `"Two Year"`  
     - **35–46 months** → `"Three Year"`  
     - **47–58 months** → `"Four Year"`  
     - **59+ months** → `"Five Year"`

4. **Merge FX rates**  
   - Each NBP row is assigned its FX tenor and merged with the Cable forward curve to pick up the appropriate USD/GBP rate.

5. **Convert units**

- NBP (p/th) is converted to usd/MMBtu:  
  $$
  \text{NBP (usd/MMBtu)} \;=\; \frac{\text{NBP (p/th)}}{10} \times \text{Cable FX rate}
  $$

- Division by 10 comes from the therm → MMBtu conversion.


6. **Final schema**  
   - Keep only the useful columns:  
     - `label` (contract)  
     - `NBP (p/th)`  
     - `Cable Fwd Rate`  
     - `NBP (usd/MMBtu)`  

7. **Integration**  
   - This transformed NBP curve is saved back to `commodity_data["NF"]`.  
   - It is then merged with Henry Hub, Brent, TTF, and JKM to form the **master flat price DataFrame**.

**Why this matters**  
- Aligns all gas benchmarks (Henry, TTF, JKM, NBP) in **usd/MMBtu** for direct comparison.  
- Ensures FX exposure is handled correctly via the forward curve, not just current spot applied across the entire curve


---
### Brent conversion and geographical arbitrage spreads (lines 317–356)

This section standardises **Brent** into natural-gas units and constructs key **geographical spreads** between LNG benchmarks.  

**1. Convert Brent to USD/MMBtu**  
Brent is originally quoted in USD/bbl. Using a standard energy equivalence of **5.8 MMBtu per barrel**:

$$
\text{Brent (USD/MMBtu)} = \frac{\text{Brent (USD/bbl)}}{5.8}
$$


**2. Geographical spreads**  
The default spread we track is **JKM – TTF**:

$$
\text{JKM v TTF} = \text{JKM (USD/MMBtu)} - \text{TTF (USD/MMBtu)}
$$

This is included in the output by default, as it represents the core **Asia vs Europe arb or East/West arb**.

**3. Optional spreads**  
Other spreads are coded but commented out by default (between lines 325–350).
To enable, simply **uncomment both the calculation and the inclusion** in the output table:

- HH v TTF → U.S. vs European gas  
- JKM v HH → Asia vs U.S. gas  
- JKM v NBP → Asia vs UK gas  
- TTF v NBP → European benchmarks vs UK gas  
- Brent (USD/MMBtu) → crude oil benchmark in gas terms  

- Some of these spreads maybe more relevant to us when our American Volume comes online, Miguel may start to ask for them.


**4. Output table (`df_merged`)**  
Final flat-price DataFrame includes:

- Core benchmarks: `JKM`, `TTF`, `NBP`, `Henry Hub`
- Default spread: `JKM v TTF`
- Brent price in both usd/bbl and (optionally) usd/MMBtu  
- FX info for NBP conversion: `NBP (p/th)`, `Cable Fwd Rate`

This DataFrame is then copied to **`df_merged_flat_price`**, which is the version exported to **Sheet 1 (Flat Prices)** in the Excel workbook.

**Note**  
For now, only **JKM v TTF** is kept live. If our LNG trader (Miguel) requests additional arbitrage views, uncomment the relevant lines mentioned above.

---
### M1/M2 spreads across the curve (lines 361–401)

**Purpose**
Measure month‑on‑month structure (contango/backwardation) for each series.

**What happens**
- Drop any `"Cash"` row (M1/M2 logic assumes monthly structure).
- Ensure the table is sorted by contract date.
- Build labels like `Aug/Sep'25`, `Sep/Oct'25`, ...
- For each target column, compute **M1 − M2** and store as `<col> Spread`.

**Targets** - line 376 add to the list if we have more curves to refer to
- `JKM (usd/MMBtu)`
- `TTF (usd/MMBtu)`
- `NBP (usd/MMBtu)`
- `Henry Hub (usd/MMBtu)`
- `JKM v TTF` (the geo spread also gets an M1/M2, but don't include unecessarily, check with Miguel if he wants them for the HH spreads as well)

**Interpretation**
- **Positive** M1−M2 → **backwardation**.
- **Negative** M1−M2 → **contango**.

**Output**
- `df_time_spreads` with:
  - `Spread Label` (e.g., `Aug/Sep'25`)
  - One `<series> Spread` column per target.

*Tip:* Uncomment the rounding line if you want 3dp in the output.

---
### Seasonal strips: Summer / Winter and the spreads between them (lines 411–486)

**Purpose**  
Build seasonal **legs** (averages) and **seasonal spreads** from the monthly curve so you can see Summer vs Winter value and how they trade against each other.


**Seasons (by delivery month)**  
- **Summer (`Sum'YY`)**: Apr–Sep of year `YY`  
- **Winter (`Win'YY`)**: Oct–Mar spanning `YY → YY+1`  


**Settlement alignment**  
A seasonal strip **settles when its first delivery month settles**:  
- `Sum'YY` settles when `Apr’YY` settles  
- `Win'YY` settles when `Oct’YY` settles  


**Key implementation detail**  
- `price_cols` (line 417) defines which series are included.  
  To add new curves into the sheet, **edit this list**.


**Code logic**  
1. **Prep monthly dates**  
   Parse labels → contract dates, add `month`/`year`, keep price columns (`JKM`, `TTF`, `NBP`, `Henry Hub`, `JKM v TTF`).  

2. **Seasonal averages**  
   Function `seasonal_avg()` groups months per season and computes the **mean** of each price column.  
   - Output labels: `Win'YY`, `Sum'YY`.  

3. **Dynamic ladder**  
   Build alternating Winter/Summer ladder forward from **today**, skipping expired strips.  
   - Winter expiry check → Sept 30 (first month Oct)  
   - Summer expiry check → Mar 31 (first month Apr)  

4. **Season-to-season spreads**  
   For each adjacent pair, compute:  

   $$
   \text{Spread} = \text{Previous season average} - \text{Next season average}
   $$  

   Example label: `Win'26 - Sum'27`.  

5. **Final output**  
   Concatenate seasonal averages (legs) and spreads into `df_seasonal_strips`.


**How to interpret**  
- **Positive Winter–Summer** → Winter premium (tight market).  
- **Negative Winter–Summer** → Summer premium.  
- Each series (`JKM`, `TTF`, `NBP`, `Henry Hub`, `JKM v TTF`) has its own seasonal legs + spreads.

**Notes**  
- Missing months = skipped season.  
- Averages are computed with `mean(skipna=True)`.

---
### Quarterly legs and Q-on-Q spreads (lines 491–601)

**Purpose**  
Aggregate monthly curves into **quarterly averages** (Q1/Q2/Q3/Q4) and build **rolling quarter-on-quarter** spreads.

**Prep**
- Copy the merged monthly table and parse `label → contract_date → quarter/month/year`.
- Keep only rows with valid dates.

**Quarter expiry & horizon**
- A **quarter expires when its first delivery month has settled** (settles end of prior month).  
  Mapping used:
  - **Q1 (Jan–Mar)** → expires end **Dec** (previous year)  
  - **Q2 (Apr–Jun)** → expires end **Mar**  
  - **Q3 (Jul–Sep)** → expires end **Jun**  
  - **Q4 (Oct–Dec)** → expires end **Sep**  
- Also drop quarters beyond a **rolling 4-year horizon** from today.  
- Result: `expired_quarters` filtered out.

**Valid quarters only**
- Keep only quarters that have **all 3 constituent months** present (`count == 3`).

**Quarterly averages (lines 539–599)**  
- Each quarter’s average is calculated for:  
  - `JKM (usd/MMBtu)`, `TTF (usd/MMBtu)`, `NBP (usd/MMBtu)`, `Henry Hub (usd/MMBtu)`, `JKM v TTF`.  
- **Important**: If you add new curves/columns to the pricing sheet (e.g. `JKM v HH` or `Brent ($/MMBtu)`), you must also add them into this `agg(...)` dictionary (in line 539) and downstream renaming block.  
- Labels are formatted `Qn'YY` (e.g., `Q1'27`), then renamed to `<series> Spread` for consistency.


**Rolling Q-on-Q spreads**
- Create a shifted copy and compute **PrevQ − CurrQ** per series:  
  $$
  \text{Q-on-Q Spread} = \text{Quarter}_{t-1} - \text{Quarter}_{t}
  $$
- Spread labels: `Qn'YY - Qm'YY` (e.g., `Q1'27 - Q2'27`).  
- First row has no previous quarter → dropped.

**Final output**
- Concatenate:
  1) **Quarterly averages** (legs), and  
  2) **Rolling Q-on-Q spreads**  
  into **`df_quarterly_strips`**.


**How to read**
- **Positive Q-on-Q** → earlier quarter trades at a premium to the next (backwardation across quarters).
- **Negative Q-on-Q** → earlier trades at a discount to the trailing (contango).


**Notes**
- If any quarter is missing a month → excluded (ensures integrity).  
- Expiry dates use a **safe day=28** fallback.  
- The 4-year horizon should be sufficient but check with trader on what he/she wants.

---
### Calendar (Cal) legs and Cal–Cal spreads (lines 603–685)

**Purpose**  
Aggregate monthly prices into **calendar‑year averages** (Cal 'YY) and build **rolling calendar spreads** (Prev Cal − Curr Cal).


**Expiry & horizon**
- A **Cal 'YY** strip **expires at the end of Dec (YY−1)**. We filter out expired years and anything beyond a rolling **5‑year horizon** from today.


**What the code does**
1. **Prep**  
   Parse `label → contract_date`, derive `month`/`year`, and keep only valid rows.

2. **Filter valid Cal years**
   - Keep years `YY` where **today ≤ 31‑Dec‑(YY−1)** and `YY ≤ today.year + 5`.

3. **Calendar averages**  
   Group by `year` and compute the mean for:
   - `JKM (usd/MMBtu)`, `TTF (usd/MMBtu)`, `NBP (usd/MMBtu)`, `Henry Hub (usd/MMBtu)`, `JKM v TTF`  
   Then label as **`Cal 'YY`** and rename columns to `<series> Spread` for a consistent schema.

4. **Keep partial data sensibly**  
   Drop only rows where **all** series are NaN (keeps partially‑populated years if at least one series has data).

5. **Cal–Cal spreads**  
   Build a shifted table and compute **Prev Cal − Curr Cal** per series:
   $$
   \text{Cal–Cal Spread} = \text{Cal}_{t-1} - \text{Cal}_{t}
   $$
   Label format: `"Cal '26 - Cal '27"`. Drop the first (no previous Cal).

6. **Final output**
   Concatenate calendar **averages** and **Cal–Cal spreads** into **`calendar_df`**.


**How to read**
- **Positive** Cal–Cal → earlier calendar trades at premium (backwardated).  
- **Negative** Cal–Cal → later calendar trades at premium (contango).


**Where to edit when adding new series**
If you introduce a new column (e.g., `JKM v HH`, `Brent ($/MMBtu)`), update all three spots:

1) **Calendar averages** (groupby list)  - line 625
```python 
cal_avg = df.groupby("year")[[
    "JKM ($/MMBtu)",
    "TTF ($/MMBtu)",
    "NBP ($/MMBtu)",
    "Henry Hub ($/MMBtu)",
    "JKM v TTF",
    # "Brent ($/MMBtu)",        # ← add here
    # "JKM v HH",               # ← add here
]].mean().round(4).reset_index()


2) **Reorder & rename block** - line 638
```python
cal_avg = cal_avg[[
    "Spread Label",
    "JKM ($/MMBtu)",
    "TTF ($/MMBtu)",
    "NBP ($/MMBtu)",
    "Henry Hub ($/MMBtu)",
    "JKM v TTF",
    # "Brent ($/MMBtu)",        # ← add here to include in output
    # "JKM v HH",
]].rename(columns={
    "JKM ($/MMBtu)": "JKM ($/MMBtu) Spread",
    "TTF ($/MMBtu)": "TTF ($/MMBtu) Spread",
    "NBP ($/MMBtu)": "NBP ($/MMBtu) Spread",
    "Henry Hub ($/MMBtu)": "Henry Hub ($/MMBtu) Spread",
    "JKM v TTF": "JKM v TTF Spread",
    # "Brent ($/MMBtu)": "Brent ($/MMBtu) Spread",  # ← add rename
    # "JKM v HH": "JKM v HH Spread",
})

3. **Cal–Cal loop columns** - line 672
```python
for col in [
    "JKM ($/MMBtu) Spread",
    "TTF ($/MMBtu) Spread",
    "NBP ($/MMBtu) Spread",
    "Henry Hub ($/MMBtu) Spread",
    "JKM v TTF Spread",
    # "Brent ($/MMBtu) Spread",  # ← add here to compute Cal–Cal
    # "JKM v HH Spread",
]:
    cal_spreads[col] = cal_avg[col].shift() - cal_avg[col]

---
### Assemble all spreads for **Sheet 2: “Spread Summary”** (lines 699–723)

**Purpose**  
Standardise the shape of each spreads table and stitch them together (with blank separators) into a single DataFrame for export.

**What this block does**
1. **Column alignment (schema guard)**
   - `align_columns_safe(df, name)` reindexes each input (`df_time_spreads`, `quarterly_df`, `df_season_output`, `calendar_df`) to the shared schema `columns_order`.
   - If a DataFrame is missing columns or has extras, it’s coerced to the exact `columns_order`.  
   - Any error logs a clear message and returns an **empty** table with the right headers.

2. **Blank spacers**
   - Builds `blank_row` (one empty row across `columns_order`) to visually separate blocks in Excel.

3. **Concatenate in display order**
   - `pd.concat([...], ignore_index=True)` stacks:
     - **Time Spreads (M1/M2)**
     - **Quarterly Strips + Q-on-Q**
     - **Seasonal Legs + Season-to-Season**
     - **Calendar Legs + Cal–Cal**
   - Each section is separated by `blank_row` for readability.

**Edit points when adding new series**
- **Update `columns_order`** to include the new `<Series> Spread` column so all four blocks align.
- Ensure each upstream block **creates** that column:
  - Time spreads builder
  - Quarterly averages & Q-on-Q loop
  - Seasonal averages & season-to-season loop
  - Calendar averages & Cal–Cal loop
- If you forget to add the new column in any block, that section will show **NaN** for the new field after reindexing (by design, not a crash).

**Why align first?**
- Keeps sheet layout stable for downstream users.
- Prevents accidental column drift when someone adds/removes a spread upstream.

**Output**
- `df_all_spreads` → a single, neatly separated table that is written to **Sheet 2: “Spread Summary”** in the Excel export step.

---
### Excel Writer & Formatting (lines 727–778)

**Purpose**  
Export the results into a final Excel file with two sheets:  
1. **Flat Prices** – monthly benchmarks 
2. **Spread Summary** – all time, seasonal, quarterly, and calendar spreads  

**Key steps**  
- File auto-named: `LNG_Pricing_Sheet_YYYY-MM-DD.xlsx`  
- Round all spread values to **3dp**  
- Apply formatting:
  - Brent/NPB → 2dp  
  - Cable FX → 4dp  
  - All other floats → 3dp  
  - Text centered  
- Auto-adjust column widths  
- Freeze top row for both sheets  

**Notes**  
If new curves are added, make sure they’re included in:  
- `df_merged_flat_price` (Flat Prices)  
- `columns_order` + spread DataFrames (Spread Summary)  
- Update formatting if decimals differ.

---
### Delivery mechanism (end of LNG script line 780 onwards)

This function emails the LNG Excel workbook (`filename`) as an attachment.  
It uses **SMTP with STARTTLS** (Office365) to securely connect and send.

**Key config (lines 840 to 855)**

- **from_email / login**  
  - Sender is `alert@irh.ae` (must match SMTP login).  
  - IRH-owned mailbox for automated reports.

- **to_email**  
  - Primary recipients (e.g., LNG trader).  
  - Example: `["Miguel.Arroyo@irh.ae"]`.

- **cc / bcc**  
  - Optional additional recipients.  
  - CC is visible, BCC is hidden.

- **smtp_server / smtp_port**  
  - Office365 → `smtp.office365.com`, port **587** (STARTTLS).

- **password**  
  - Loaded from environment: `EMAIL_PASSWORD`.  
  - For now, this is entered **directly on line 13** of  
    `lng-pricing-automation/.github/workflows/pricing_runner.yml`.  
  - The value should be a Microsoft 365 **App Password** generated for the `alert@irh.ae` account.  
  - Storing passwords directly in YAML is not best practice — once possible, migrate this into a GitHub **Secret** and reference it in the workflow for security.

**Notes**  
- Subject typically: `LNG Pricing Sheet – {today_str}`.  
- Update `to_email` / `cc` / `bcc` lists as distribution changes.  
- If IT rotates credentials, update the GitHub Secret only—no code changes needed.
